# Get review data and teams keywords

In [139]:
REVIEW_PATH = '../Resources/1_clean_review_data'

KEYWORDS_PATH = '../Resources/2_keywords_for_teams_data'

DATA_PATH = '../Resources'

REVIEW_COL = 'Review'
TITLE_COL = 'Title'

In [140]:
data = pd.read_csv(os.path.join(REVIEW_PATH,'cleanAppStore.csv'))

In [141]:
with open(os.path.join(KEYWORDS_PATH,'team_keywords.json'),'r') as f:
    team_names_keywords = json.load(f)

with open(os.path.join(KEYWORDS_PATH,'grouped_team_keywords.json'),'r') as f:
    groups_keywords = json.load(f)
    
with open(os.path.join(KEYWORDS_PATH,'grouped_team_mapping.json'),'r') as f:
    mapping = json.load(f)

# Extract keywords in 2 ways

## Term_extractor (POS-tagging based algorithm)

In [142]:
from rutermextract import TermExtractor
term_extractor = TermExtractor()

In [143]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [144]:
reviews_keywords = {}
empty_index = []
for i in tqdm(data.index):
    reviews_keywords[i] = []
    for term in term_extractor(data.iloc[i][REVIEW_COL]):
        reviews_keywords[i].append(term.normalized)
    for term in term_extractor(data.iloc[i][TITLE_COL]):
        reviews_keywords[i].append(term.normalized)
    reviews_keywords[i] = list(set(reviews_keywords[i]))
    if len(reviews_keywords[i])==0:
        del reviews_keywords[i]
        empty_index.append(i)

100%|██████████| 11584/11584 [00:53<00:00, 218.31it/s]


In [150]:
reviews_keywords

{0: ['сбербанк', 'приложение'],
 1: ['обновление', 'неделя', 'мои друзья', 'приложение'],
 2: ['функция',
  'работа',
  'последнее обновление',
  'обновлённая версия',
  'приложение',
  'сканирование лица',
  'добрый день'],
 3: ['подключение wi-fi'],
 4: ['минус', 'applepay', 'баланс', 'mastercard'],
 5: ['перевод',
  'нея',
  'номер телефона',
  'деньга',
  'поддержка',
  'банка',
  'аккаунт',
  'карта',
  '2 карты',
  'проблема',
  'такая низкая оценка',
  'такой крупный банка',
  'приложение',
  'ошибка',
  'тупой бот',
  'нонсенс'],
 6: ['история', 'возможность'],
 7: ['2020 год',
  'вид',
  'ваши жёлтые конкуренты',
  'банка',
  'сии поры',
  'доставка',
  'отдетение',
  'ваше банка',
  'огромная база зарплатников',
  'все счёт',
  'банк',
  'сбер',
  'одно город',
  'другие банка',
  'ребята',
  'офис',
  'пенсионеры',
  'переводы',
  'карта',
  'закрой',
  'другое отделение'],
 8: ['перевод денег', 'банка'],
 9: ['дипломированные мошенники'],
 10: ['половина операций',
  'прило

In [159]:
data.iloc[3][REVIEW_COL]

'Не могу скачать так как требует подключения Wi-fi'

## TextRank Gensim

In [147]:
from gensim.summarization import keywords
import re

In [148]:
reviews_keywords_gensim = {}
for i in tqdm(data.index):
    reviews_keywords_gensim[i] = []
    text = data.iloc[i][REVIEW_COL]
    lemm = []
    for word in text.split(u' '):
        if re.match(u'([^a-zа-яe ̈]+)', word):
            word = morph.parse(word)[0].normal_form
        lemm.append(word)
    text = ' '.join(lemm)
    reviews_keywords_gensim[i]+=keywords(text).split('\n')
    text = data.iloc[i][TITLE_COL]
    lemm = []
    for word in text.split(u' '):
        if re.match(u'([^a-zа-яe ̈]+)', word):
            word = morph.parse(word)[0].normal_form
            lemm.append(word)
    text = ' '.join(lemm)
    reviews_keywords_gensim[i]+=keywords(text).split('\n')
#     reviews_keywords_gensim[i] = list(set(reviews_keywords_gensim[i]))

100%|██████████| 11584/11584 [00:28<00:00, 407.65it/s]


In [149]:
reviews_keywords_gensim

{0: ['стала', ''],
 1: ['пытаюсь', ''],
 2: ['день', 'функция', ''],
 3: ['требует', ''],
 4: ['через', ''],
 5: ['деньги',
  'связаться',
  'что',
  'крупном банке',
  'как',
  'приоритетная',
  'проблему',
  ''],
 6: ['дико', ''],
 7: ['карту',
  'внутри банка',
  'банки',
  'базе',
  'доставка',
  'переводы',
  'тем',
  'вид',
  'ребят',
  ''],
 8: ['почему', ''],
 9: ['', ''],
 10: ['раньше', 'время половину', ''],
 11: ['супер', ''],
 12: ['', ''],
 13: ['как', 'без', ''],
 14: ['быстрое', ''],
 15: ['интерфеисе', 'время', ''],
 16: ['быть', ''],
 17: ['остаток', 'бюджет', ''],
 18: ['батарея', ''],
 19: ['онлаин', ''],
 20: ['', ''],
 21: ['само', 'один', ''],
 22: ['переводить', 'как', 'только', 'тыс', 'бесчеловечныи', 'сосать', ''],
 23: ['работает', 'телефоне', ''],
 24: ['ежемесячнои', 'этим', 'банковское', 'своего', 'повторить', ''],
 25: ['выводить', 'где', 'равно', ''],
 26: ['тоже', 'история', 'переходов', ''],
 27: ['', ''],
 28: ['версия',
  'для планшета это',
  'буду'

## Conclusion: 

Gensim works worse for keywords extracting than library

# Common words baseline

In [151]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from numpy import argmax
import sklearn.preprocessing
import numpy as np

## One hot encoding

In [153]:
def transform(review_keywords, groups_keywords):
    tokens = []
    for key in review_keywords:
        tokens+=review_keywords[key]
    for key in groups_keywords:
        tokens+=groups_keywords[key]
    print()
    print('All Tokens:')
    print(len(tokens))

    results = []
    label_enc = sklearn.preprocessing.LabelEncoder()
    onehot_enc = sklearn.preprocessing.OneHotEncoder()
    
    encoded_all_tokens = label_enc.fit_transform(list(set(tokens)))
    encoded_all_tokens = encoded_all_tokens.reshape(len(encoded_all_tokens), 1)
    
    onehot_enc.fit(encoded_all_tokens)
    
    review_keywords_transformed = {}
    for key in tqdm(review_keywords):
        
        encoded_words = label_enc.transform(review_keywords[key])
        
        
        encoded_words = onehot_enc.transform(encoded_words.reshape(len(encoded_words), 1))
        

        review_keywords_transformed[key]=(np.mean(encoded_words.toarray(), axis=0))
        
    groups_keywords_transformed = {}
    for key in tqdm(groups_keywords):
        
        encoded_words = label_enc.transform(groups_keywords[key])
        
        
        encoded_words = onehot_enc.transform(encoded_words.reshape(len(encoded_words), 1))
        

        groups_keywords_transformed[key]=(np.mean(encoded_words.toarray(), axis=0))
    
    return review_keywords_transformed, groups_keywords_transformed

## Groups classification 

In [79]:
review_keywords_transformed, groups_keywords_transformed = transform(reviews_keywords, groups_keywords)

  0%|          | 11/11139 [00:00<01:46, 104.53it/s]


All Tokens:
65731


100%|██████████| 26/26 [00:00<00:00, 107.68it/s]


In [101]:
predictions = {}

for key in tqdm(review_keywords_transformed):
    predictions[key] = {}
    for group in groups_keywords_transformed:
        score = euclidean_distances([groups_keywords_transformed[group]], [review_keywords_transformed[key]])[0][0]
        predictions[key][group] = score

100%|██████████| 11139/11139 [00:52<00:00, 211.49it/s]


In [105]:
import random
from collections import Counter,OrderedDict

#sampling with replacement
sampling = random.choices(list(reviews_keywords.keys()), k=20)

### Eucleadean distance

In [118]:
for sample in sampling:
    print('---')
    print('Review')
    print(data.iloc[sample][TITLE_COL], data.iloc[sample][REVIEW_COL])
    print()
    scores = OrderedDict(sorted(predictions[sample].items(), key=lambda kv: kv[1]))
    print('Nearest team')
    print(list(scores.keys())[0], scores[list(scores.keys())[0]])
    print('---')

---
Review
Стало очень громоздким Отличное приложение, прекрасно выполняет свои функции, но с годами становится все более и более громоздким. Последнее обновление не несло ничего по сути кроме украшений а весило больше 250мб. Лично у меня на телефоне места всегда не хватает.. Эти открытки и сторис мне лично не нужны, я была бы счастлива отключить и скрыть их и тем освободить немного места на телефоне, пока что удалось в настройках скрыть часть не нужного мне контента на главном экране приложения, но на занимаемый объем памяти это никак не повлияло.. Сейчас это одно из самых громоздких приложений, но оно необходимо так что приходится терпеть.
Была бы безмерно благодарна за лайт-версию без всяких поющих отрыток, стикеров, сторисов, и тому подобного а только с базовыми функциями или за возможность отключить и скрыть их в текущей версии освободив тем немного места. Думаю я не одна такая..

Nearest team
Госуслуги 5.0
---
---
Review
Третий день не работает ! Третий день не работает ! Возможн

### Conclusion:
Not weighted eucleadean distance predict best scores for groups with the smallest number of keywords (Госуслуги)

In [109]:
predictions_cos = {}

for key in tqdm(review_keywords_transformed):
    predictions_cos[key] = {}
    for group in groups_keywords_transformed:
        score = cosine_similarity([groups_keywords_transformed[group]], [review_keywords_transformed[key]])[0][0]
        predictions_cos[key][group] = score

100%|██████████| 11139/11139 [01:30<00:00, 122.74it/s]


In [119]:
for sample in sampling:
    print('---')
    print('Review')
    print(data.iloc[sample][TITLE_COL], data.iloc[sample][REVIEW_COL])
    print()
    scores = OrderedDict(sorted(predictions_cos[sample].items(), key=lambda kv: kv[1], reverse = True))
    print('Nearest team')
    print(list(scores.keys())[0], scores[list(scores.keys())[0]])
    print('---')

---
Review
Стало очень громоздким Отличное приложение, прекрасно выполняет свои функции, но с годами становится все более и более громоздким. Последнее обновление не несло ничего по сути кроме украшений а весило больше 250мб. Лично у меня на телефоне места всегда не хватает.. Эти открытки и сторис мне лично не нужны, я была бы счастлива отключить и скрыть их и тем освободить немного места на телефоне, пока что удалось в настройках скрыть часть не нужного мне контента на главном экране приложения, но на занимаемый объем памяти это никак не повлияло.. Сейчас это одно из самых громоздких приложений, но оно необходимо так что приходится терпеть.
Была бы безмерно благодарна за лайт-версию без всяких поющих отрыток, стикеров, сторисов, и тому подобного а только с базовыми функциями или за возможность отключить и скрыть их в текущей версии освободив тем немного места. Думаю я не одна такая..

Nearest team
интерфейс приложения 0.0
---
---
Review
Третий день не работает ! Третий день не работае

### Conclusion:
Cosine distance predict scores as zero as number of keywords in groups is small, but predicts "карты" not bad

# Cosine distance for teams

In [154]:
review_keywords_transformed, teams_keywords_transformed = transform(reviews_keywords, team_names_keywords)

  0%|          | 8/11139 [00:00<02:28, 74.71it/s]


All Tokens:
65764


100%|██████████| 42/42 [00:00<00:00, 110.13it/s]


In [156]:
predictions_cos_teams = []

for key in tqdm(review_keywords_transformed):
    predictions_cos_teams.append({})
    predictions_cos_teams[-1]['review'] = data.iloc[key][TITLE_COL]+' '+data.iloc[key][REVIEW_COL]
    scores = {}
    for team in teams_keywords_transformed:
        score = cosine_similarity([teams_keywords_transformed[team]], [review_keywords_transformed[key]])[0][0]
        scores[team] = score
    scores = OrderedDict(sorted(scores.items(), key=lambda kv: kv[1], reverse = True))
    predictions_cos_teams[-1]['label']=list(scores.keys())[0]

100%|██████████| 11139/11139 [02:33<00:00, 72.79it/s]


In [ ]:
pd.DataFrame(predictions_cos_teams).to_csv(os.path.join(DATA_PATH,'tfidf_cos_preds.csv'))

In [131]:
predictions_cos_teams

{0: {'iOS Platform': 0.0,
  'Global Navigation': 0.0,
  'DDA Profile': 0.0,
  'PFM': 0.0,
  'PFMMPLACE': 0.0,
  'PUSH IOS': 0.0,
  'DBP.Витрины продаж': 0.0,
  'Госуслуги': 0.0,
  'iOS Release Engineer': 0.0,
  'Мессенджер': 0.0,
  'store-n-sales': 0.0,
  'Integration Platform': 0.0,
  'Data Driven App': 0.0,
  'Краудфандинг': 0.0,
  'DBP.Подарки': 0.0,
  'Sberbank ID B2C': 0.0,
  'История операций': 0.0,
  'Редактируемый профиль клиента': 0.0,
  'ЕФС Выписки и справки. Mobile': 0.0,
  'ЕФС.Платежи МП': 0.0,
  'Платежи. Штрафы': 0.0,
  'ПДВ в Digital': 0.0,
  'СБОЛ. Классические переводы': 0.0,
  'ЕФС.Автоплатежи': 0.0,
  'Автопереводы': 0.0,
  'Digital Сбербанк Премьер': 0.0,
  'Развитие лояльности в МП СБОЛ': 0.0,
  'Дебетовые карты в мобильном приложении': 0.0,
  'Карта в телефоне': 0.0,
  'Плановый и досрочный перевыпуск дебетовых карт': 0.0,
  'Самозанятые': 0.0,
  'Цифровой Кредит': 0.0,
  'ВС.МП вклады': 0.0,
  'Комиссионные продукты': 0.0,
  'PFM Бюджет': 0.0,
  'Mobile Online 

In [130]:
for sample in sampling:
    scores = OrderedDict(sorted(predictions_cos_teams[sample].items(), key=lambda kv: kv[1], reverse = True))
    if scores[list(scores.keys())[0]]!=0:
        print('---')
        print('Review')
        print(data.iloc[sample][TITLE_COL], data.iloc[sample][REVIEW_COL])
        print()
        print('Nearest team')
        print(list(scores.keys())[0], scores[list(scores.keys())[0]])
        print('---')

---
Review
Баг Внизу на вкладке «диалоги» постоянно висит уведомление о непрочитанных диалогах, хотя их там нет. Когда открываешь вкладку, уведомление исчезает, после повторного входа в приложение появляется снова

Nearest team
Sberbank ID B2C 0.16903085094570333
---
---
Review
Куда делись раздел расход по счету Пропала в начале страницы расход по карте ??????

Nearest team
Плановый и досрочный перевыпуск дебетовых карт 0.13608276348795434
---
---
Review
Загрузка Сделайте приложение меньшим размером. Требуется подключение к WiFi, так как файл весит более 200 мб.

Nearest team
Global Navigation 0.07412493166611012
---
---
Review
Нет отображения истории платежей.(((( Спасибо, исправили.

Nearest team
iOS Platform 0.10783277320343841
---
---
Review
Проблема есть Не работает оплата по штрих-коду и QR-коду! Не найден лицевой счет!

Nearest team
ЕФС.Платежи МП 0.08944271909999157
---
---
Review
Выбор карты для платежей «по умолчанию» Буквально сегодня обнаружили, что сменилась карта по умолч